In [ ]:
import chess
import chess.pgn
from sklearn.linear_model import LinearRegression
import numpy as np


In [ ]:
PGN = "../madchess-v-viri.pgn"
# list of (phase, move) tuples
xs: "list[tuple[int, int]]" = []
# list of moves-left values
ys: "list[int]" = []
pawn_phase, knight_phase, bishop_phase, rook_phase, queen_phase = [1, 10, 10, 20, 40]

In [ ]:
game_counter = 0
with open(PGN, "r") as pgn:
    while True:
        pos_counter = 0
        game = chess.pgn.read_game(pgn)
        if game is None:  # EOF
            break
        game_counter += 1
        nodes = game.mainline()
        for node in nodes:
            board = node.board()
            phase = 0
            phase += chess.popcount(board.pawns) * pawn_phase
            phase += chess.popcount(board.knights) * knight_phase
            phase += chess.popcount(board.bishops) * bishop_phase
            phase += chess.popcount(board.rooks) * rook_phase
            phase += chess.popcount(board.queens) * queen_phase
            xs.append((phase, board.ply()))
            pos_counter += 1
        for i in range(pos_counter):
            ys.append(pos_counter - i)
    assert len(xs) == len(ys), f"{len(xs) = }, {len(ys) = }"
print(f"Read {game_counter} games, {len(xs)} positions")

In [ ]:
# augment xs with features
xs_aug = [ [x[0], x[1], x[0] * x[1], x[0] ** 2, x[1] ** 2] for x in xs ]

In [ ]:
# fit model
model = LinearRegression()
model.fit(xs, ys)
# write model
weights = model.coef_
intercept = model.intercept_
print(f"{weights = }")
print(f"{intercept = }")
mse = np.mean((model.predict(xs) - ys) ** 2)
print(f"{mse = }")

In [ ]:
# fit augmented model
model_aug = LinearRegression()
model_aug.fit(xs_aug, ys)
# write model
weights_aug = list(model_aug.coef_)
intercept_aug = model_aug.intercept_
print(f"{weights_aug = }")
print(f"{intercept_aug = }")
mse_aug = np.mean((model_aug.predict(xs_aug) - ys) ** 2)
print(f"{mse_aug = }")

In [ ]:
# test the MSE of always predicting 30
mse_30 = np.mean((30 - np.array(ys)) ** 2)
print(f"{mse_30 = }")

In [ ]:
# average model output over all positions
avg = sum(ys) / len(ys)
print(f"{avg = }")

In [ ]:
1 / 30.0 * avg

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import LinearLocator
from matplotlib import cm

In [ ]:
# plot a 3D surface
fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
# Make data.
X = np.arange(0, 256, 1)
Y = np.arange(0, 400, 1)
X, Y = np.meshgrid(X, Y)
Z = weights[0] * X + weights[1] * Y + intercept
# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.coolwarm,
                          linewidth=0, antialiased=False)
# Customize the z axis.
ax.set_zlim(0, 400)
ax.zaxis.set_major_locator(LinearLocator(10))
# Add a colour bar which maps values to colours.
fig.colorbar(surf, shrink=0.5, aspect=5)
plt.show()